In [2]:
import pandas as pd
import numpy as np
import sklearn
import datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq

from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
from surprise import Reader, Dataset
from surprise import KNNBaseline
from surprise.model_selection import GridSearchCV

from surprise import SVD, BaselineOnly, CoClustering

from tqdm import tqdm
import pickle


Подготовка тестового датасета

In [ ]:
train = pq.read_table('train_without_None.gzip').to_pandas()
dict_cookie_to_user={i:j for i,j in zip(train.cookie_id, train.user_id_filled)}


In [6]:
test_public_mfti = pq.read_table('test_public_mfti.parquet').to_pandas()
test_public_mfti['user_id']=test_public_mfti.cookie_id.apply(lambda x: dict_cookie_to_user[x])

In [3]:
list_vacansy_user_saw=[]
for i in tqdm(range(len(test_public_mfti))):
    list_vacansy_user_saw.append(set(train.loc[train["user_id_filled"] == test_public_mfti.user_id[i], "vacancy_id_"]))
    

100%|██████████| 772/772 [10:23<00:00,  1.24it/s]


In [5]:
test_public_mfti['list_vacansy_user_saw']=list_vacansy_user_saw
test_public_mfti.to_pickle('test_public_mfti_add_vacansy_user_saw.pickle')



In [3]:
test_public_mfti = pd.read_pickle('test_public_mfti_add_vacansy_user_saw.pickle')
test_public_mfti

,cookie_id,vacancy_id_,user_id,list_vacansy_user_saw
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...",477e719bfbcc4071a8acc1b357492b00,"{189576, 156688, 114834, 104082, 213013, 13442..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...",753a2a268ae84ab49475e62736c02860,"{240258, 195726, 116494, 176398, 139539, 21698..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...",d01c76b282364fbb8195f326de3e893c,"{195716, 141447, 191628, 206350, 117520, 25652..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]",6adb4495090b454395d5aa2f1a5ff9d9,"{187652, 248331, 203404, 198393, 184726, 18038..."
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]",b107696e3ff54e8a83182828bb845bc6,"{167168, 148864, 254087, 153353, 101389, 11726..."
...,...,...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]",da74810d481449278a75a091178f9831,"{229314, 212971, 103567, 170333, 243294}"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212...",4da39c1e048d49cabcd1229fdcc6413f,"{181381, 140678, 198407, 108295, 253194, 21287..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]",5207e9855ec54e02aa8cfa48ede76e1b,"{219841, 119075, 108093, 109990, 244646, 19575..."
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052...",7f2067ab30994440b13a0ea37a5e8001,"{102794, 117134, 217615, 236821, 130587, 23403..."


In [37]:
new_column=[]

for i in tqdm(range(len(test_public_mfti))):
    
    user_saw=list(test_public_mfti['list_vacansy_user_saw'][i])

    all_same_user=list(train.user_id_filled[train.vacancy_id_.isin(user_saw)])

    all_vacansy_saw_same_user=list(train.vacancy_id_[train.user_id_filled.isin(all_same_user)])
    
    new_column.append(set(all_vacansy_saw_same_user))
    
    
    

    


100%|██████████| 772/772 [11:44<00:00,  1.10it/s]


In [38]:
test_public_mfti['all_vacansy_saw_same_user']=new_column

test_public_mfti.to_pickle('test_public_mfti_add_all_vacansy_saw_same_user.pickle')



In [21]:
test_public_mfti=pd.read_pickle('test_public_mfti_add_all_vacansy_saw_same_user.pickle')
test_public_mfti

,cookie_id,vacancy_id_,user_id,list_vacansy_user_saw,all_vacansy_saw_same_user
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812...",477e719bfbcc4071a8acc1b357492b00,"{189576, 156688, 114834, 104082, 213013, 13442...","{131073, 131079, 131081, 131085, 131087, 13108..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348...",753a2a268ae84ab49475e62736c02860,"{240258, 176398, 195726, 116494, 139539, 21698...","{100001, 100002, 100003, 100006, 100007, 10000..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065...",d01c76b282364fbb8195f326de3e893c,"{195716, 141447, 191628, 206350, 117520, 25652...","{100001, 100002, 100003, 100006, 100007, 10000..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]",6adb4495090b454395d5aa2f1a5ff9d9,"{187652, 248331, 203404, 184726, 180382, 15824...","{100001, 100002, 100003, 100005, 100006, 10000..."
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]",b107696e3ff54e8a83182828bb845bc6,"{167168, 148864, 254087, 153353, 101389, 11726...","{131073, 131079, 131081, 131082, 131088, 13108..."
...,...,...,...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]",da74810d481449278a75a091178f9831,"{229314, 212971, 103567, 170333, 243294}","{196609, 196611, 229382, 196626, 196628, 22940..."
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212...",4da39c1e048d49cabcd1229fdcc6413f,"{181381, 140678, 108295, 198407, 253194, 21287...","{131074, 131085, 131088, 131122, 131135, 13114..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]",5207e9855ec54e02aa8cfa48ede76e1b,"{219841, 119075, 208350, 244646, 109990, 19575...","{131073, 131074, 131077, 131078, 131079, 13108..."
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052...",7f2067ab30994440b13a0ea37a5e8001,"{102794, 117134, 217615, 236821, 130587, 23403...","{131073, 131074, 131081, 131085, 131088, 13108..."


Анализ тестового дата сета

In [23]:
train_group = pd.read_pickle('train_group_add_cookie.pickle')

In [24]:
user_id=train_group.index.get_level_values(0)
vacancy_id_=train_group.index.get_level_values(1)
data = pd.DataFrame(list(zip( user_id, vacancy_id_, train_group['event_type'].to_list())), columns = ['user_id', 'vacancy_id_', 'rating'])

data

,user_id,vacancy_id_,rating
0,000089d26cdd49d68839c68bc10f2cf2,177380,1
1,000089d26cdd49d68839c68bc10f2cf2,181353,1
2,000089d26cdd49d68839c68bc10f2cf2,195890,1
3,000089d26cdd49d68839c68bc10f2cf2,205562,1
4,000089d26cdd49d68839c68bc10f2cf2,211064,1
...,...,...,...
4662685,ffffdb17f45b4032b386d691d52e6c00,107125,10
4662686,ffffdb17f45b4032b386d691d52e6c00,107893,10
4662687,ffffdb17f45b4032b386d691d52e6c00,150843,10
4662688,ffffdb17f45b4032b386d691d52e6c00,160164,10


Добавим столбец пересения vacancy_id_ и all_vacansy_saw_same_user

In [66]:
new_column=[]

for i in tqdm(range(len(test_public_mfti))):
     new_column.append(set(test_public_mfti['vacancy_id_'][i]) & test_public_mfti['all_vacansy_saw_same_user'][i])
    

100%|██████████| 772/772 [00:00<00:00, 29949.62it/s]


In [67]:
test_public_mfti['intersection']=new_column

Проанализируем данные в этом столбце:

In [68]:
test_public_mfti['intersection']

0      {222177, 225604, 119314, 235544, 238874, 22812...
1      {221376, 257319, 240744, 235561, 102794, 11348...
2      {197505, 207108, 245637, 203015, 234380, 24065...
3       {235196, 210628, 212325, 138123, 115420, 246685}
4               {214561, 251469, 166899, 219070, 212703}
                             ...                        
767                     {213344, 222146, 207851, 214180}
768    {232128, 229677, 109360, 230546, 259570, 23759...
769             {253946, 114852, 251081, 244688, 127546}
770    {100103, 240521, 236363, 220747, 244688, 17956...
771             {256719, 100336, 244721, 230803, 116442}
Name: intersection, Length: 772, dtype: object

In [69]:
test_public_mfti['len_intersection']=test_public_mfti['intersection'].apply(lambda x: len(x))

In [101]:
test_public_mfti['len_vacancy_id_']=test_public_mfti['vacancy_id_'].apply(lambda x: len(x))

In [145]:
test_public_mfti['len_all_vacansy_saw_same_user']=test_public_mfti['all_vacansy_saw_same_user'].apply(lambda x: len(x))

In [154]:
test_public_mfti['len_all_vacansy_saw_same_user'].max()

136228

In [155]:
test_public_mfti['len_all_vacansy_saw_same_user'].min()

328

Найдем вакансии, по которым количество пользователей которые совершили положительные действия, находяться выше среднего значения действий по всем вакансиям похожих пользователей

In [191]:
new_column_1=[] 
new_column_2=[] 
new_column_3=[]
new_column_4=[]
for i in tqdm(range(len(test_public_mfti))):
    # Все действия с вакансиями которые представлены в тестовом датасете, были сделаны в train
    all_do_with_vac=data.loc[data.vacancy_id_.isin(test_public_mfti['intersection'][i])]
    # Подсчитаем по каждой вакансии всех пользователей которые сделали хотя бы одно полдожительное действие 10
    vacancy_ids_count_10=np.unique (np.asarray(all_do_with_vac.vacancy_id_[all_do_with_vac.rating>1]), return_counts= True )
    
    #Аналогично для всех вакансий похожих пользователей
    all_do_with_vac_all=data.loc[data.vacancy_id_.isin(test_public_mfti['all_vacansy_saw_same_user'][i])]
    
    vacancy_ids_count_all_10=np.unique (np.asarray(all_do_with_vac_all.vacancy_id_[all_do_with_vac_all.rating>1]), return_counts= True )
    
    #Среднее количество пользователей для всех похожих вакансий
    m=np.mean(vacancy_ids_count_all_10[1])
    
    #Вакансии больше среднего значения
    
    new_column_1.append(vacancy_ids_count_10[0][vacancy_ids_count_10[1]>m])
    
    #Значение 75%
    
    b=np.mean(vacancy_ids_count_all_10[1][vacancy_ids_count_all_10[1]>np.mean(vacancy_ids_count_all_10[1])])
    
    new_column_2.append(vacancy_ids_count_10[0][vacancy_ids_count_10[1]>b])
    
    # значение топ 20 вакансий
    
    top_20=np.mean(sorted(vacancy_ids_count_all_10[1])[-20:])
    
    new_column_3.append(vacancy_ids_count_10[0][vacancy_ids_count_10[1]>top_20])
    
    # значение топ 50 вакансий
    
    top_50=np.mean(sorted(vacancy_ids_count_all_10[1])[-50:])
    
    new_column_4.append(vacancy_ids_count_10[0][vacancy_ids_count_10[1]>top_50])
    

100%|██████████| 772/772 [04:02<00:00,  3.19it/s]


In [192]:
test_public_mfti['vacancy_more_50_%']=new_column_1
test_public_mfti['vacancy_more_75_%']=new_column_2
test_public_mfti['vacancy_more_top_20']=new_column_3
test_public_mfti['vacancy_more_top_50']=new_column_4

In [195]:
test_public_mfti['len_vacancy_more_50_%']=test_public_mfti['vacancy_more_50_%'].apply(lambda x: len(x))
test_public_mfti['len_vacancy_more_75_%']=test_public_mfti['vacancy_more_75_%'].apply(lambda x: len(x))
test_public_mfti['len_vacancy_more_top_20']=test_public_mfti['vacancy_more_top_20'].apply(lambda x: len(x))
test_public_mfti['len_vacancy_more_top_50']=test_public_mfti['vacancy_more_top_50'].apply(lambda x: len(x))

In [ ]:
test_public_mfti.to_pickle('test_public_mfti_analysis.pickle')


In [3]:
test_public_mfti = pd.read_pickle('test_public_mfti_analysis.pickle')

In [4]:
test_public_mfti[test_public_mfti['len_vacancy_more_top_20']>0]

,cookie_id,vacancy_id_,user_id,list_vacansy_user_saw,all_vacansy_saw_same_user,intersection,len_intersection,len_vacancy_id_,len_all_vacansy_saw_same_user,vacancy_more_50_%,vacancy_more_75_%,len_vacancy_more_50_%,len_vacancy_more_75_%,vacancy_more_top_20,len_vacancy_more_top_20,vacancy_more_top_50,len_vacancy_more_top_0,len_vacancy_more_top_50
12,043b9752285e491ea4d34807dcc94065,"[207423, 240649, 116651, 119608, 221920, 21530...",35b4791e864541d2b3e615800d1d0016,"{230145, 108174, 206350, 213150, 217894, 14403...","{100001, 100002, 100003, 100006, 100007, 10001...","{221920, 240649, 215305, 116651, 230707, 11960...",8,8,83745,"[207423, 215305, 221920, 230707, 237341]","[207423, 215305, 221920, 230707, 237341]",5,5,[207423],1,[207423],1,1
36,0c9bc6e28eb74a65890ab8f35587f7c8,"[232857, 182084, 203404, 207423, 230419]",413ed9d6426c438e912f2d5ee6db9dba,"{231553, 253315, 171013, 228230, 258441, 16218...","{100002, 100003, 100006, 100007, 100010, 10001...","{182084, 203404, 230419, 232857, 207423}",5,5,83720,"[182084, 203404, 207423, 230419, 232857]","[182084, 203404, 207423, 230419, 232857]",5,5,[207423],1,"[203404, 207423]",2,2
58,13ca760a7b3948d0aa48adbed26f3b67,"[239744, 226061, 182870, 220718, 110793, 22760...",f0fff61c53cb4bc0a18431e6cd02bb04,"{205184, 111617, 203404, 111505, 111383, 23734...","{100001, 100002, 100003, 100005, 100007, 10000...","{239744, 238691, 110793, 226061, 220718, 23530...",9,9,107083,"[110793, 182870, 220718, 232245, 238691, 239744]","[110793, 182870, 220718, 232245]",6,4,[182870],1,[182870],1,1
74,18604a58ee7d49d3a75780f2b1999238,"[230707, 257698, 201370, 182870, 110578, 22808...",8198e80f910943e5a3695af9682fb0e4,"{242560, 121475, 203404, 246413, 206350, 23054...","{100001, 100002, 100003, 100005, 100007, 10000...","{257698, 220718, 110578, 230707, 228084, 14955...",9,9,100754,"[149557, 182870, 220718, 230707]","[149557, 182870, 220718, 230707]",4,4,[182870],1,[182870],1,1
75,186cae2dd58a42bb965fce95ff93e6d5,"[232857, 207108, 228866, 228253, 240695, 16460...",e39bf14d1e9142db9103afb5d0cf5a01,"{174723, 182084, 219948, 180398, 202608, 11186...","{100001, 100002, 100003, 100004, 100007, 10000...","{228866, 207108, 230405, 102919, 203404, 23041...",19,19,81971,"[102919, 115420, 125867, 149557, 164602, 20340...","[102919, 115420, 125867, 149557, 164602, 20340...",17,14,[207423],1,"[164602, 203404, 207423]",3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,f3b9473d542a4b4284ed7aeb1d1100c9,"[234893, 236510, 235946, 238691, 108345, 19999...",d8ef0a671b7040f0aeb71764e1ae6169,"{206211, 207108, 162187, 203404, 149387, 11150...","{100001, 100002, 100003, 100004, 100005, 10000...","{240544, 238691, 236602, 236766, 240135, 23594...",18,18,131556,"[108345, 148714, 199990, 220718, 230425, 23489...","[108345, 148714, 199990, 220718, 230425, 23489...",16,9,[148714],1,[148714],1,1
734,f3ff3a77eff94997ad7b15e82e0c9976,"[238046, 211189, 207108, 210503, 203404, 11183...",47ac9706851842f19b958026dc435573,"{183466, 117963, 180398, 139446, 160668}","{131122, 131146, 131173, 131180, 131198, 13121...","{207108, 212325, 210503, 237341, 203404, 23070...",11,12,20805,"[111837, 203404, 207108, 207423, 210503, 21118...","[111837, 203404, 207108, 207423, 212325, 23070...",11,8,[207423],1,"[203404, 207423]",2,2
741,f5c67ff429f64b4b9478e577432a6a3c,"[240614, 240075, 107369, 223446, 230707, 23489...",78af4077e18d4158808e3b06ceb61463,"{239624, 162315, 100876, 248852, 166938, 10141...","{100001, 100002, 100003, 100005, 100006, 10000...","{240614, 107369, 240075, 125867, 234893, 24065...",9,9,126876,"[107369, 125867, 207423, 223446, 230707, 23489...","[107369, 125867, 207423, 223446, 230707, 234893]",7,6,[207423],1,[207423],1,1
747,f71abd9d3c9c41ce8db075767d9c3494,"[182870, 198413, 205010, 212325, 230906, 23173...",9d036b4276fc430ca8fda22b58a14f87,"{165441, 198114, 189345, 127352, 233389, 20635...","{100001, 100002, 100003, 100005, 100007, 10000...","{149024, 212325, 237288, 198413, 205010, 23173...",9,9,97865

Только для 69 пользователей из тестового датасета могут быть полученны значения свыше 20% по метрике предложенной заказчиком

А максимальная возможная точность составит в %:

In [11]:
print(f"точность по метрике заказчика {round(sum(test_public_mfti['len_vacancy_more_top_20'])*20/772, 2)} % для топ 20 вакансий") 


точность по метрике заказчика 1.99 % для топ 20 вакансий


И для топ 50:

In [12]:
print(f"точность по метрике заказчика {round(sum(test_public_mfti['len_vacancy_more_top_50'])*20/772, 2)} % для топ 50 вакансий") 


точность по метрике заказчика 4.12 % для топ 50 вакансий


In [206]:
test_public_mfti.to_pickle('test_public_mfti_analysis.pickle')